# Text Classification
## This notebook outlines the usage of NLP Feature extraction (CountVectorizer, TfidfVectorizer) in classification of text documents

### Import all the necessary libraries

In [1]:
from pprint import pprint
from time import time
import logging

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

### Choose a few categories fro the entire 20 categories

In [2]:
# Load some categories from the training set
categories = [
    'alt.atheism',
    'talk.religion.misc',
]

In [3]:
print("Loading 20 newsgroups dataset for categories:")
print(categories)

Loading 20 newsgroups dataset for categories:
['alt.atheism', 'talk.religion.misc']


### Fetch documents for these 2 categories

In [4]:
data = fetch_20newsgroups(subset='train', categories=categories)
print(f"{len(data.filenames)} documents")
print(f"{len(data.target_names)} categories")
print()

857 documents
2 categories



### Define a pipeline combining a text feature extractor with a simple classifier

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(tol=1e-3)),
])

### Specify parameter grid
- 'vect__max_df': (0.5, 0.75, 1.0)
- 'vect__max_features': (None, 5000, 10000, 50000)
- 'vect__ngram_range': ((1, 1), (1, 2))
- 'tfidf__use_idf': (True, False)
- 'tfidf__norm': ('l1', 'l2')
- 'clf__max_iter': (20,)
- 'clf__alpha': (0.00001, 0.000001)
- 'clf__penalty': ('l2', 'elasticnet')
- 'clf__max_iter': (10, 50, 80)

In [6]:
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    # 'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    # 'tfidf__use_idf': (True, False),
    # 'tfidf__norm': ('l1', 'l2'),
    'clf__max_iter': (20,),
    'clf__alpha': (0.00001, 0.000001),
    'clf__penalty': ('l2', 'elasticnet'),
    # 'clf__max_iter': (10, 50, 80),
}

### Find the best parameters for both the feature extraction and the classifier

### Build a GridSearch with the pipeline and parameter grid

In [7]:
grid_search = GridSearchCV(pipeline, parameters, cv=5,
                           n_jobs=-1, verbose=1)

### Start the grid search

In [9]:
grid_search.fit(data.data, data.target)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', SGDClassifier())]),
             n_jobs=-1,
             param_grid={'clf__alpha': (1e-05, 1e-06), 'clf__max_iter': (20,),
                         'clf__penalty': ('l2', 'elasticnet'),
                         'vect__max_df': (0.5, 0.75, 1.0),
                         'vect__ngram_range': ((1, 1), (1, 2))},
             verbose=1)

### Best Score

In [10]:
print("Best score: %0.3f" % grid_search.best_score_)

Best score: 0.953


### Best Parameter

In [11]:
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best parameters set:
	clf__alpha: 1e-05
	clf__max_iter: 20
	clf__penalty: 'l2'
	vect__max_df: 1.0
	vect__ngram_range: (1, 2)


 ### Make Word2Vec and Doc2Vec work (Credit: Stackoverflow)

In [14]:
pip install nltk


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.0/291.0 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 9.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [15]:
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from nltk.tokenize import word_tokenize

class Word2VecTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, model):
        self.model = model

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return [np.maximum(self.model.wv[word], 0) if word in self.model.wv else np.zeros(self.model.vector_size) for word in X]

class Doc2VecTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, model):
        self.model = model

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return [np.maximum(self.model.infer_vector(word_tokenize(doc)), 0) if len(word_tokenize(doc)) > 0 else np.zeros(self.model.vector_size) for doc in X]


In [17]:
pip install gensim


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 20.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [20]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /Users/maple/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [21]:
import numpy as np
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec, Doc2Vec
from gensim.models.doc2vec import TaggedDocument

# Assuming 'data.data' contains your text data
word2vec_model = Word2Vec(sentences=[word_tokenize(text) for text in data.data])

tagged_data = [TaggedDocument(words=word_tokenize(doc), tags=[str(i)]) for i, doc in enumerate(data.data)]
doc2vec_model = Doc2Vec()
doc2vec_model.build_vocab(tagged_data)
doc2vec_model.train(tagged_data, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)


### Algo and Extractor

In [24]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer


algorithms = [
    ('Multinomial Naïve Bayes', MultinomialNB()),
    ('Logistic Regression', LogisticRegression()),
    ('Support Vector Machines', SVC()),
    ('Decision Trees', DecisionTreeClassifier()),
]

extractors = [
    ('Bag-of-Words', CountVectorizer()),
    ('N-grams', CountVectorizer(ngram_range=(1, 2))),
    ('word2vec', Word2VecTransformer(model=word2vec_model)),
    ('doc2vec', Doc2VecTransformer(model=doc2vec_model))
]


### Combine pipeline, Set parameters = all (avoid error), do gridsearch, and benchmarking (performance, and accuracy).

In [25]:
results = []

for algo_name, algo in algorithms:
    for extractor_name, extractor in extractors:
        pipeline = Pipeline([
            ('vect', extractor),
            ('clf', algo),
        ])

        parameters = {}

        grid_search = GridSearchCV(pipeline, parameters, cv=5,
                                   n_jobs=-1, verbose=1)

        start_time = time()
        grid_search.fit(data.data, data.target)
        end_time = time()

        execution_time = end_time - start_time
        best_score = grid_search.best_score_
        best_parameters = grid_search.best_estimator_.get_params()

        results.append({
            'Algorithm': algo_name,
            'Feature Extractor': extractor_name,
            'Best Score': best_score,
            'Execution Time': execution_time,
            'Best Parameters': best_parameters
        })

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits


/Users/maple/anaconda3/envs/T431/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/maple/anaconda3/envs/T431/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mo

Fitting 5 folds for each of 1 candidates, totalling 5 fits


/Users/maple/anaconda3/envs/T431/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/maple/anaconda3/envs/T431/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mo

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits


### Save in tabulate form (Credit: ChatGPT)

In [27]:
pip install tabulate


Note: you may need to restart the kernel to use updated packages.


In [28]:
from tabulate import tabulate

# Store the best configuration
best_configuration = max(results, key=lambda x: x['Best Score'])

# Display the best configuration
print("Best:")
pprint(best_configuration)

# Save the results to a text or document file
with open('results.txt', 'w') as file:
    # Write the best configuration
    file.write("Best:\n")
    file.write(tabulate([best_configuration], headers='keys', tablefmt='grid'))

    # Write all configurations
    file.write("\n\nAll Possibles:\n")
    file.write(tabulate(results, headers='keys', tablefmt='grid'))

Best:
{'Algorithm': 'Logistic Regression',
 'Best Parameters': {'clf': LogisticRegression(),
                     'clf__C': 1.0,
                     'clf__class_weight': None,
                     'clf__dual': False,
                     'clf__fit_intercept': True,
                     'clf__intercept_scaling': 1,
                     'clf__l1_ratio': None,
                     'clf__max_iter': 100,
                     'clf__multi_class': 'auto',
                     'clf__n_jobs': None,
                     'clf__penalty': 'l2',
                     'clf__random_state': None,
                     'clf__solver': 'lbfgs',
                     'clf__tol': 0.0001,
                     'clf__verbose': 0,
                     'clf__warm_start': False,
                     'memory': None,
                     'steps': [('vect', CountVectorizer(ngram_range=(1, 2))),
                               ('clf', LogisticRegression())],
                     'vect': CountVectorizer(ngram_range=(1, 2)